# Costruzioni dataset

In [5]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

## Dataset per RDF2Text

### Train e dev set

In [6]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]
dataset = []

i = 0
for dataset_type in dataset_types:
    for triple_number in triple_numbers:
        path = "..\\..\\..\\..\\webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                i += 1
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                for entry in root.iter('entry'):
                    mts = []
                    eid = entry.get('eid')
                    category = entry.get('category')
                    size = entry.get('size')

                    if entry.get('shape_type') is not None:
                        shape_type = entry.get('shape_type')
                    else:
                        shape_type = "NA"

                    for modifiedtripleset in entry.iter('modifiedtripleset'):
                    #for modifiedtripleset in entry.iter('originaltripleset'):
                        mtriples = ""

                        triples = []
                        for mtriple in modifiedtripleset.iter('mtriple'):  
                        #for mtriple in modifiedtripleset.iter('otriple'):  
                            triple = mtriple.text.split(" | ")
                            triple = (triple[0], triple[1], triple[2])
                            triples.append(triple)

                        triples.sort(key=lambda x: x[1])
                        for triple in triples:
                            mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "

                        # if last character is space remove it
                        if mtriples[-1] == " ":
                            mtriples = mtriples[:-1]

                        mts.append(mtriples)

                    lexs = []
                    for lex in entry.iter('lex'):
                        if lex.get('lang') == 'it-PE':
                            lexs.append(lex.text)

                    for mt in mts:
                        i += 1
                        dataset.append([eid, size, dataset_type, category, shape_type, mt, lexs])

### Test set

In [7]:
path = "..\\..\\..\\..\\webnlg\\it-PE\\test"
file_names = []
dataset_type = "test"

for file_name in os.listdir(path):
    if os.path.isfile(os.path.join(path, file_name)) and file_name == "rdf-to-text-generation-test-data-with-refs-en.xml":
        i += 1
        url = os.path.join(path, file_name)
        tree = ET.parse(url)
        root = tree.getroot()

        for entry in root.iter('entry'):
            mts = []
            eid = entry.get('eid')
            category = entry.get('category')
            size = entry.get('size')

            if entry.get('shape_type') is not None:
                shape_type = entry.get('shape_type')
            else:
                shape_type = "NA"

            for modifiedtripleset in entry.iter('modifiedtripleset'):
            #for modifiedtripleset in entry.iter('originaltripleset'):
                mtriples = ""

                triples = []
                for mtriple in modifiedtripleset.iter('mtriple'):  
                #for mtriple in modifiedtripleset.iter('otriple'):  
                    triple = mtriple.text.split(" | ")
                    triple = (triple[0], triple[1], triple[2])
                    triples.append(triple)

                triples.sort(key=lambda x: x[1])
                for triple in triples:
                    mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "

                # if last character is space remove it
                if mtriples[-1] == " ":
                    mtriples = mtriples[:-1]

                mts.append(mtriples)

            lexs = []
            for lex in entry.iter('lex'):
                if lex.get('lang') == 'it-PE':
                    lexs.append(lex.text)

            for mt in mts:
                i += 1
                dataset.append([eid, size, dataset_type, category, shape_type, mt, lexs])

In [8]:
# create a file called ge_dataset.csv
df = pd.DataFrame(dataset, columns = ['eid', 'size', 'dataset_type', 'category', 'shape_type', 'data_unit', 'sentence'])
df.to_csv("it_dataset.csv", index=False)